In [1]:
!pip install weblinx
!pip install huggingface_hub[cli]
!pip install openai
!pip install datasets

In [2]:
from datetime import datetime
import json
import os
import time
from pathlib import Path
import weblinx as wl
from pathlib import Path

# Weblinx data preparation

In [3]:
with open("splits.json") as f:
    splits= json.load(f)

In [ ]:
from huggingface_hub import snapshot_download
from datasets import load_dataset

#demo_names = ['saabwsg', 'ygprzve', 'iqaazif']
demo_names = splits['train'] #'test_vis', 'test_geo', 'test_cat', 'test_web', 'iid_all', 'train', 'valid', 'test_iid'
patterns = [f"demonstrations/{name}/*.json*" for name in demo_names]
#patterns = ["*.json*"]  
snapshot_download(
    "McGill-NLP/WebLINX-full",repo_type="dataset", local_dir="./wl_data", allow_patterns=patterns, etag_timeout =400
)



# GPT API and Weblinx utils

In [3]:
def load_json_no_cache(basedir, name):
    if not os.path.exists(f"{basedir}/{name}.json"):
        return None
    try:
        with open(f"{basedir}/{name}.json", "r") as f:
            j = json.load(f)
    except:
        return None
    
    return j


def load_recording(basedir):
    # Before loading replay, we need a dropdown that allows us to select replay.json or replay_orig.json
    # Find all files in basedir starting with "replay" and ending with ".json"
    replay_files = sorted(
        [
            f
            for f in os.listdir(basedir)
            if f.startswith("replay") and f.endswith(".json")
        ]
    )
    replay_file = 'replay.json'
    replay_file = replay_file.replace(".json", "")

    metadata = load_json_no_cache(basedir, "metadata")


    # Read in the JSON data
    replay_dict = load_json_no_cache(basedir, replay_file)


    data = replay_dict["data"]
    return data,metadata

def format_chat_message(d):
    if d["speaker"] == "instructor":
        return "🧑 " + d["utterance"]
    else:
        return "🤖 " + d["utterance"]

In [4]:
from os import listdir
from os.path import isfile, join


def create_prompt(mainPrompt,mainResponse,prompt_dir):
    onlyfiles = [f for f in listdir(prompt_dir) if isfile(join(prompt_dir, f))]
    len_mes= int((len(onlyfiles)-1)/2)
    messages=[{"role": "user", "content": mainPrompt}, {"role": "assistant", "content": mainResponse}]
    for i in range(len_mes):
        question = open(f'{prompt_dir}/question{i}.txt', "r").read()    
        answer = open(f'{prompt_dir}/answer{i}.txt', "r").read()
        messages.append({"role": "user", "content": question})
        messages.append({"role": "assistant", "content": answer})

    return messages
    


In [5]:
import json
import re
import openai
from openai import OpenAI


class ParseError(Exception):
    pass


def extract_html_tags(text, keys):
    """Extract the content within HTML tags for a list of keys.

    All text and keys will be converted to lowercase before matching.

    Args:
        text (str): The input string containing the HTML tags.
        keys (list[str]): The HTML tags to extract the content from.

    Returns:
        dict: A dictionary mapping each key to a list of subset in `text` that match the key.
    """
    content_dict = {}
    # text = text.lower()
    # keys = set([k.lower() for k in keys])
    for key in keys:
        pattern = f"<{key}>(.*?)</{key}>"
        matches = re.findall(pattern, text, re.DOTALL)
        if matches:
            content_dict[key] = [match.strip() for match in matches]
    return content_dict

def parse_html_tags(text, keys=(), optional_keys=(), merge_multiple=False):
    """Satisfy the parse api, extracts 1 match per key and validates that all keys are present

    Args:
        text (str): The input string containing the HTML tags.
        keys (list[str]): The HTML tags to extract the content from.
        optional_keys (list[str]): The HTML tags to extract the content from, but are optional.
        merge_multiple (bool): Whether to merge multiple instances of the same key.

    Returns:
        dict: A dictionary mapping each key to a subset of `text` that match the key.
        bool: Whether the parsing was successful.
        str: A message to be displayed to the agent if the parsing was not successful.

    """
    all_keys = tuple(keys) + tuple(optional_keys)
    content_dict = extract_html_tags(text, all_keys)
    retry_messages = []

    for key in all_keys:
        if not key in content_dict:
            if not key in optional_keys:
                retry_messages.append(f"Missing the key <{key}> in the answer.")
        else:
            val = content_dict[key]
            content_dict[key] = val[0]
            if len(val) > 1:
                if not merge_multiple:
                    retry_messages.append(
                        f"Found multiple instances of the key {key}. You should have only one of them."
                    )
                else:
                    # merge the multiple instances
                    content_dict[key] = "\n".join(val)

    valid = len(retry_messages) == 0
    retry_message = "\n".join(retry_messages)
    return content_dict, valid, retry_message

def parse_html_tags_raise(text, keys=(), optional_keys=(), merge_multiple=False):
    """A version of parse_html_tags that raises an exception if the parsing is not successful."""
    content_dict, valid, retry_message = parse_html_tags(
        text, keys, optional_keys, merge_multiple=merge_multiple
    )
    if not valid:
        raise ParseError(retry_message)
    return content_dict

def adapt_text(text, n_retry):
  client = OpenAI(api_key=os.environ.get("OPENAI"))
  tries= 0
  while tries < n_retry:
    try:
      prompt_dir= 'prompts/general'
      mainPrompt= open(f'{prompt_dir}/mainPrompt.txt').read()
      mainResponse= open(f'{prompt_dir}/mainResponse.txt').read()
      messages = create_prompt(mainPrompt,mainResponse,prompt_dir)
      messages.append({"role": "user", "content": text})
      response = client.chat.completions.create(
                    model="gpt-3.5-turbo-0125",
                    temperature=0.4,
                    messages = messages
                )
      reply_content = response.choices[0].message.content
      ans_dict = parse_html_tags_raise(reply_content,keys=('goal','steps') )
      print(ans_dict)
      return ans_dict
    except ParseError as parsing_error:
            tries += 1

    raise ParseError(f"Could not parse a valid value after {n_retry} retries.")
        


  return reply_content 

# Goal and Step abstraction from Weblinx

The following methods abstract demonstrations from Weblinx related to a certain URL and with GPT's API extract a general goal and steps.

In [6]:
demo_names = [f for f in os.listdir('wl_data/demonstrations') ] #demo names available

In [ ]:
import time

wl_dir = Path("./wl_data")
base_dir = wl_dir / "demonstrations"
split_path = "splits.json"
urls= ["https://www.amazon.com/", "https://www.ebay.com/", "https://www.wikipedia.org/", "https://www.encyclopedia.com/", "https://www.reddit.com/"]
#focused_url=    "https://www.amazon.com/"  #Select URL to filter demonstrations  #MAKE GOOGLE MAPS EXAMPLES
#urls = ["https://www.amazon.com/"]

for focused_url in urls:
    # Filter demonstrations based on focused_url
    demos_with_url=set()
    demos = [wl.Demonstration(name, base_dir=base_dir) for name in demo_names]
    for demo in demos:
        replay = wl.Replay.from_demonstration(demo)
        turns = replay.filter_by_intents("load")
        if len(turns) == 0:
            continue
        for turn in turns:
            if focused_url in turn.url:
                print(demo.name)
                demos_with_url.add(demo.name)
    #time.sleep(2)
    # Extract step and goals based on demonstration
    prompt_dir= f'prompts/{focused_url.split('.')[1]}'
    demos = [wl.Demonstration(name, base_dir=base_dir) for name in demos_with_url]
    for i,demo in enumerate(demos):
        replay = wl.Replay.from_demonstration(demo)
        turns = replay.filter_by_type("chat")
        if len(turns) == 0:
            continue
        stringBuilder= ""
        for turn in turns:    
            stringBuilder+=(format_chat_message(turn))+'\n'
        print(stringBuilder)
        steps_goal= adapt_text(stringBuilder,3)
        with open(f'{prompt_dir}/question{i}.txt', 'w+') as f:
            f.write(steps_goal['goal'])
        with open(f'{prompt_dir}/answer{i}.txt', 'w+') as f:
            f.write(steps_goal['steps'])
        print('--------------------------------------------------------')
        #time.sleep(3)


kjhbavb
kxmdswx
nhltddf
kdyvkxj
lsnfixr
wrkuqxa
mwywyzg
nlxbjoy
ikmvfow
🧑 Hi
🤖 Hello
🧑 Open the Trello website.
🤖 Yes, sure.
🧑 Create a workspace.
🤖 Okay.
🤖 What would be the workspace name?
🧑 Stellar Solutions
🤖 Please tell me the workspace type.
🧑 Engineering-IT
🤖 Would you like to add a Workspace description [Optional]?
🧑 Yes, add the Workspace Description as follows: 
	 At Stellar Solutions, we believe in empowering businesses to reach new heights through innovative and efficient workspace solutions. Our workspace is designed to inspire creativity, foster collaboration, and provide a nurturing environment for your team to thrive.
🤖 Would you like to invite your team members?
🧑 Yes. Here are the email IDs: 
	 esytest5@gmail.com, abcdef@gmail.com, qwerty@gmail.com
🤖 Done. Anything else you would like to do?
🧑 No, we are done here.
🤖 Okay.

{'goal': '"Create a workspace on Trello named \'Stellar Solutions\' with the type \'Engineering-IT\', including a workspace description and inviti

# Inference (step prediction based on goal)
Here we are testing the few-shot capability of creating good plans

### 1. Reddit

In [92]:
import json
import re
import openai
from openai import OpenAI

def get_steps(text, n_retry=3):
  client = OpenAI(api_key=os.environ.get("OPENAI"))
  tries= 0
  while tries < n_retry:
    try:
      prompt_dir= 'prompts/reddit'
      mainPrompt= open(f'{prompt_dir}/mainPrompt.txt').read()
      mainResponse= open(f'{prompt_dir}/mainResponse.txt').read()
      messages = create_prompt(mainPrompt,mainResponse,prompt_dir)
      messages.append({"role": "user", "content": f"Create steps for the goal:{text}"})
      response = client.chat.completions.create(
                    model="gpt-3.5-turbo-0125",
                    temperature=0.2,
                    messages = messages
                )
      reply_content = response.choices[0].message.content

      return reply_content
    except ParseError as parsing_error:
            tries += 1

    raise ParseError(f"Could not parse a valid value after {n_retry} retries.")
        
  return reply_content 

In [93]:
print (get_steps("Tell me the count of comments that have received more downvotes than upvotes for the user who made the latest post on the Showerthoughts forum."))

1. Open Reddit and navigate to the Showerthoughts forum.
2. Identify the latest post made on the forum.
3. Click on the post to view the user who made the post.
4. Check the comments section of the post.
5. For each comment, calculate the number of downvotes and upvotes.
6. Count the comments where the downvotes are greater than the upvotes.
7. Provide the user with the count of comments that meet this criteria.


In [ ]:
print(get_steps("Among the top 10 post in \"books\" forum, show me the post URLs that recommand a single book"))

1. Open Reddit.
2. Navigate to the "books" subreddit.
3. Locate the top 10 posts.
4. Identify the posts that recommend a single book.
5. Provide the URLs of those specific posts.


In [ ]:
print(get_steps('Change my reddit bio to "I am a robot"'))

1. Open Reddit.
2. Go to your profile.
3. Find the section for editing your bio.
4. Change your bio text to "I am a robot."
5. Save the changes.


In [ ]:
print(get_steps("Upvote the newest post in books subreddit"))

1. Open Reddit.
2. Navigate to the "Books" subreddit.
3. Sort the posts by "Newest" to find the latest post.
4. Upvote the newest post in the "Books" subreddit.
